<div align=left style="width: 200; height: 80px; overflow: hidden">
    <img src=http://static1.squarespace.com/static/571446ff60b5e92c3a2b4249/57d8a40b9de4bb459f731cf3/58cb2f229de4bb4a049d38c2/1505340359463/teselaGenlogo.jpg align=right width=200>
</div>

# Hello World! DESIGN Module

In [1]:
import platform
from pathlib import Path
import pandas as pd
import json

import fastaparser
from dna_features_viewer import GraphicFeature, GraphicRecord

from teselagen.api.design_client import DESIGNClient

print(f"python version     : {platform.python_version()}")
print(f"pandas version     : {pd.__version__}")

python version     : 3.6.9
pandas version     : 1.1.5


In [2]:
client = DESIGNClient()
#client = DESIGNClient(host_url="https://platform.teselagen.com")
# The following command will promt you to type username (email) and password
client.login()
# Alternatively you can add credential information in the arguments
#client.login(username="your@email", password="ZZZ")

Connection Refused


In [3]:
#designs_info = pd.DataFrame(client.get_designs())
#display(designs_info)

In [4]:
#sequences = pd.DataFrame(client.get_dna_sequences(name='pj5_001'))
#display(sequences)

In [5]:
#sequence = client.get_dna_sequence(seq_id=237629, out_format='fasta')



One way to upload a design into DESIGN module is by using the json format. In the file [design_example.json](design_example.json) there is a very simple design that can be visualized like this:

<div align=left style="width: 400; overflow: hidden">
    <img align=center width=400, src="data:image/png;base64, ...">
</div>

We can upload this design to DESIGN very easily. For this, we first select a lab and then invoke the `post_design` method.

In [6]:
client.select_laboratory("Python")
with open('design_example.json') as json_file:
    design = json.load(json_file)
    response = client.post_design(design=design)
    display(response)

Exception: Got code : 502. Reason : Bad Gateway